In [3]:
import pandas as pd 
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

In [4]:
data= pd.read_csv("D:\PFE\Fact_Table_Rev.csv", low_memory=False)

In [5]:
data

,montant_HT,montant_ttc,date_saisie,date_facture,Fk_prestation,Fk_famille_prestation,Fk_nationnalite,Fk_Societe,Fk_typesociete
0,4.0,3.571429,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
1,6.0,5.357143,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
2,8.0,7.142857,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
3,10.0,8.928571,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
4,8.5,7.589286,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
...,...,...,...,...,...,...,...,...,...
937562,10.5,9.375000,2013-09-24,2013-09-24,BOI,BAR,LI,GRINPARAPH,LAB
937563,7.0,6.250000,2013-09-24,2013-09-24,BOI,BAR,LI,GRINPARAPH,LAB
937564,3.5,3.125000,2013-09-24,2013-09-24,BOIR,CAVE,TN,STANDART,SOC
937565,196.0,175.000000,2013-09-24,2013-09-24,REP,RESTO,TN,STANDART,SOC


In [6]:
df = pd.DataFrame(data, columns=["montant_HT", "date_saisie", "Fk_famille_prestation","Fk_typesociete"])

In [7]:
df['date_saisie'] = pd.to_datetime(df['date_saisie'])

In [8]:
df

,montant_HT,date_saisie,Fk_famille_prestation,Fk_typesociete
0,4.0,2006-09-18,BAR,SOC
1,6.0,2006-09-18,BAR,SOC
2,8.0,2006-09-18,BAR,SOC
3,10.0,2006-09-18,BAR,SOC
4,8.5,2006-09-18,BAR,SOC
...,...,...,...,...
937562,10.5,2013-09-24,BAR,LAB
937563,7.0,2013-09-24,BAR,LAB
937564,3.5,2013-09-24,CAVE,SOC
937565,196.0,2013-09-24,RESTO,SOC


In [9]:
print(df)

        montant_HT date_saisie Fk_famille_prestation Fk_typesociete
0              4.0  2006-09-18                   BAR            SOC
1              6.0  2006-09-18                   BAR            SOC
2              8.0  2006-09-18                   BAR            SOC
3             10.0  2006-09-18                   BAR            SOC
4              8.5  2006-09-18                   BAR            SOC
...            ...         ...                   ...            ...
937562        10.5  2013-09-24                   BAR            LAB
937563         7.0  2013-09-24                   BAR            LAB
937564         3.5  2013-09-24                  CAVE            SOC
937565       196.0  2013-09-24                 RESTO            SOC
937566         3.5  2013-09-24                  CAVE            PET

[937567 rows x 4 columns]


In [10]:
df= df.drop(index=df.loc[(df['date_saisie'] < '2006-01-01') | (df['date_saisie'] > '2019-12-31')].index)

In [11]:
df= df.sort_values(by='date_saisie', ascending=True)

In [10]:
print(df)

        montant_HT date_saisie Fk_famille_prestation Fk_typesociete
0              4.0  2006-09-18                   BAR            SOC
313            0.0  2006-09-18                   BAR            SOC
314            0.0  2006-09-18                   BAR            SOC
315            0.0  2006-09-18                   BAR            SOC
316            0.0  2006-09-18                   BAR            SOC
...            ...         ...                   ...            ...
765368       120.0  2019-12-31                TORRET            SOC
765367       147.0  2019-12-31                TORRET            SOC
765366        16.0  2019-12-31                TORRET            SOC
765364        44.0  2019-12-31                TORRET            SOC
765337        12.0  2019-12-31                TORRET            SOC

[753932 rows x 4 columns]


In [12]:
label_encoder = LabelEncoder()
df['Fk_prestation_code'] = label_encoder.fit_transform(df['Fk_famille_prestation'])
label_encoder = LabelEncoder()
df['Fk_typesociete_code'] = label_encoder.fit_transform(df['Fk_typesociete'])
df.head()

,montant_HT,date_saisie,Fk_famille_prestation,Fk_typesociete,Fk_prestation_code,Fk_typesociete_code
0,4.0,2006-09-18,BAR,SOC,2,5
313,0.0,2006-09-18,BAR,SOC,2,5
314,0.0,2006-09-18,BAR,SOC,2,5
315,0.0,2006-09-18,BAR,SOC,2,5
316,0.0,2006-09-18,BAR,SOC,2,5


In [12]:
df

,montant_HT,date_saisie,Fk_famille_prestation,Fk_typesociete,Fk_prestation_code,Fk_typesociete_code
0,4.0,2006-09-18,BAR,SOC,2,5
313,0.0,2006-09-18,BAR,SOC,2,5
314,0.0,2006-09-18,BAR,SOC,2,5
315,0.0,2006-09-18,BAR,SOC,2,5
316,0.0,2006-09-18,BAR,SOC,2,5
...,...,...,...,...,...,...
765368,120.0,2019-12-31,TORRET,SOC,19,5
765367,147.0,2019-12-31,TORRET,SOC,19,5
765366,16.0,2019-12-31,TORRET,SOC,19,5
765364,44.0,2019-12-31,TORRET,SOC,19,5


In [460]:
df1 = df[['Fk_famille_prestation', 'Fk_prestation_code']].drop_duplicates()
print(df1)

       Fk_famille_prestation  Fk_prestation_code
314                      BAR                   2
324                     CAVE                   3
210                    RESTO                  15
339                   SEJOUR                  17
26497                    PTD                  14
224                       MB                   9
1719                     LIN                   6
520                      TEL                  18
2416                    DEBO                   4
33558                    P.S                  10
12757                  LOCAT                   7
96329                    DIV                   5
9208                    PISC                  12
18717                   PEIG                  11
94129                 BALNEO                   0
140920                S.COIF                  16
136211                    PR                  13
208095                   LOY                   8
416562                BANQUE                   1
456896              

Linear Regression Model

In [461]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Supposons que vous avez vos données dans un DataFrame pandas appelé 'data'

# Séparation des features et de la target
X = df[['Fk_prestation_code', 'Fk_typesociete_code']]
y = df['montant_HT']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)


In [374]:
model = LinearRegression()

In [375]:
model.fit(X_train, y_train)

LinearRegression()

In [376]:
y_pred = model.predict(X_test)

In [377]:
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 90084.10141799343


In [402]:
X_test

,Fk_prestation_code,Fk_typesociete_code
727644,16,5
727643,16,5
727645,16,5
727940,16,5
727647,16,5
...,...,...
765325,12,4
765322,12,5
765492,12,5
765498,12,4



Linear Regression 2


In [13]:
X_train = df[(df['date_saisie'].dt.year >= 2006) & (df['date_saisie'].dt.year <= 2018)][['Fk_prestation_code']]
X_test = df[df['date_saisie'].dt.year == 2019][['Fk_prestation_code']]
y_train = df[(df['date_saisie'].dt.year >= 2006) & (df['date_saisie'].dt.year <= 2018)]['montant_HT']
y_test = df[df['date_saisie'].dt.year == 2019]['montant_HT']

In [472]:
model = LinearRegression()

In [473]:
model.fit(X_train, y_train)

LinearRegression()

In [474]:
y_pred = model.predict(X_test)

In [475]:
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 138449.61117165224


In [476]:
y_pred.sum()

4831987.399383133

In [477]:
result_df = pd.DataFrame({'y_pred': y_pred, 'Fk_prestation_code': X_test['Fk_prestation_code']})
sum_by_prestation_code = result_df.groupby('Fk_prestation_code')['y_pred'].sum()


In [478]:
mapping_df = df[['Fk_prestation_code', 'Fk_famille_prestation']].drop_duplicates()

# Merge total_montant_HT_by_code with mapping_df on Fk_prestation_code
total_montant_HT_with_famille = sum_by_prestation_code.reset_index().merge(mapping_df, on='Fk_prestation_code')

# Display the result
print(total_montant_HT_with_famille)

    Fk_prestation_code        y_pred Fk_famille_prestation
0                    1  1.053749e+02                BANQUE
1                    2  1.062049e+05                   BAR
2                    3  9.759502e+04                  CAVE
3                    4  3.360634e+02                  DEBO
4                    5  5.374443e+01                   DIV
5                    6  7.468102e+04                   LIN
6                    8  5.337238e+02                   LOY
7                    9  1.970513e+05                    MB
8                   12  1.712340e+04                  PISC
9                   14  7.584563e+05                   PTD
10                  15  1.088685e+06                 RESTO
11                  17  1.050454e+06                SEJOUR
12                  18  1.124439e+04                   TEL
13                  19  1.429464e+06                TORRET


In [133]:
total_montant_HT_with_famille.to_csv('total_montant_HT_with_famille.csv', index=False)

Random Forest Regressor Model


In [37]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor()

In [38]:
y_pred = rf.predict(X_test)

In [39]:
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 135036.26844967436


In [40]:
y_pred.sum()

4468619.940700877

In [41]:
result_df = pd.DataFrame({'y_pred': y_pred, 'Fk_prestation_code': X_test['Fk_prestation_code']})
sum_by_prestation_code = result_df.groupby('Fk_prestation_code')['y_pred'].sum()
mapping_df = df[['Fk_prestation_code', 'Fk_famille_prestation']].drop_duplicates()

# Merge total_montant_HT_by_code with mapping_df on Fk_prestation_code
Predfamilleprestation= sum_by_prestation_code.reset_index().merge(mapping_df, on='Fk_prestation_code')

# Display the result
print(Predfamilleprestation)

    Fk_prestation_code        y_pred Fk_famille_prestation
0                    1  7.250650e+04                BANQUE
1                    2  1.335587e+05                   BAR
2                    3  1.624293e+05                  CAVE
3                    4  1.896823e+03                  DEBO
4                    5  3.132009e+02                   DIV
5                    6  2.849942e+04                   LIN
6                    8  2.421989e+04                   LOY
7                    9  1.493479e+04                    MB
8                   12  6.781964e+03                  PISC
9                   14  2.042801e+05                   PTD
10                  15  1.860863e+06                 RESTO
11                  17  1.299857e+06                SEJOUR
12                  18  5.242257e+02                   TEL
13                  19  6.579546e+05                TORRET


In [42]:
Predfamilleprestation.to_csv('Predfamilleprestation.csv', index=False)

SARIMAX

In [13]:
df.set_index('date_saisie', inplace=True)

In [14]:
index_type = type(df.index)

# Check if it's a DateTimeIndex
is_datetime_index = isinstance(df.index, pd.DatetimeIndex)

# Print the result
print("Index type:", index_type)
print("Is DateTimeIndex:", is_datetime_index)

Index type: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
Is DateTimeIndex: True


In [15]:
df.head()

,montant_HT,Fk_famille_prestation,Fk_typesociete,Fk_prestation_code,Fk_typesociete_code
date_saisie,,,,,
2006-09-18,4.0,BAR,SOC,2,5
2006-09-18,0.0,BAR,SOC,2,5
2006-09-18,0.0,BAR,SOC,2,5
2006-09-18,0.0,BAR,SOC,2,5
2006-09-18,0.0,BAR,SOC,2,5


In [16]:
df.tail()

,montant_HT,Fk_famille_prestation,Fk_typesociete,Fk_prestation_code,Fk_typesociete_code
date_saisie,,,,,
2019-12-31,120.0,TORRET,SOC,19,5
2019-12-31,147.0,TORRET,SOC,19,5
2019-12-31,16.0,TORRET,SOC,19,5
2019-12-31,44.0,TORRET,SOC,19,5
2019-12-31,12.0,TORRET,SOC,19,5


In [17]:
selected_columns = ['montant_HT','Fk_prestation_code']
df3 = df.loc[:, selected_columns]

In [18]:
print(df3)

             montant_HT  Fk_prestation_code
date_saisie                                
2006-09-18          4.0                   2
2006-09-18          0.0                   2
2006-09-18          0.0                   2
2006-09-18          0.0                   2
2006-09-18          0.0                   2
...                 ...                 ...
2019-12-31        120.0                  19
2019-12-31        147.0                  19
2019-12-31         16.0                  19
2019-12-31         44.0                  19
2019-12-31         12.0                  19

[753932 rows x 2 columns]


In [19]:
train_data = df3.loc['2015':'2018']
test_data = df3.loc['2019':]

In [20]:
train_data

,montant_HT,Fk_prestation_code
date_saisie,,
2015-01-01,20.0,14
2015-01-01,180.0,17
2015-01-01,4.0,2
2015-01-01,50.0,15
2015-01-01,4.0,2
...,...,...
2018-12-31,20.0,9
2018-12-31,13.0,6
2018-12-31,10.0,3


In [38]:
test_data['montant_HT'].sum()

4854624.2639

In [21]:
model = SARIMAX(train_data['montant_HT'],  order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
result = model.fit() 

d:\PFE\libraires\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\PFE\libraires\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [22]:
start = len(train_data)
end = len(train_data) + len(test_data) -1
predictions = result.predict(start, end).sum()
predictions

d:\PFE\libraires\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
d:\PFE\libraires\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


5151122.773998283

In [30]:
predictions_df_reset_index = predictions.reset_index(drop=True)

In [31]:
test_data_reset_index = test_data.reset_index()

In [32]:
result_df = pd.DataFrame({'predictions': predictions_df_reset_index, 'Fk_prestation_code': test_data_reset_index['Fk_prestation_code']})

In [33]:
sum_predictions_by_code = result_df.groupby('Fk_prestation_code')['predictions'].sum()
print(sum_predictions_by_code)

Fk_prestation_code
1       2263.097622
2     834040.811609
3     469590.702609
4       1188.234539
5        184.791820
6     166131.410015
8        913.059904
9     285539.346569
12     18462.574119
14    694218.607518
15    927498.110357
17    786178.962328
18      7980.122631
19    956932.942357
Name: predictions, dtype: float64


In [28]:
rmse = mean_squared_error(test_data['montant_HT'], predictions, squared=False)
print("RMSE:", rmse)

RMSE: 373.0887389977464


d:\PFE\libraires\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


LSTM


In [35]:
df

,montant_HT,date_saisie,Fk_famille_prestation,Fk_typesociete
0,4.0,2006-09-18,BAR,SOC
313,0.0,2006-09-18,BAR,SOC
314,0.0,2006-09-18,BAR,SOC
315,0.0,2006-09-18,BAR,SOC
316,0.0,2006-09-18,BAR,SOC
...,...,...,...,...
765368,120.0,2019-12-31,TORRET,SOC
765367,147.0,2019-12-31,TORRET,SOC
765366,16.0,2019-12-31,TORRET,SOC
765364,44.0,2019-12-31,TORRET,SOC


In [37]:
selected_columns = ['date_saisie', 'montant_HT']
df1 = df.loc[:, selected_columns]
df1['year'] = df['date_saisie'].dt.year.astype(int)
df1['month'] = df['date_saisie'].dt.month.astype(int)
selected_columns = ['year', 'montant_HT']
df1 = df1.loc[:, selected_columns]
df1

,year,montant_HT
0,2006,4.0
313,2006,0.0
314,2006,0.0
315,2006,0.0
316,2006,0.0
...,...,...
765368,2019,120.0
765367,2019,147.0
765366,2019,16.0
765364,2019,44.0


In [39]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [40]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df1)

In [43]:
X = scaled_data[:, 0]  
y = scaled_data[:, 1] 

In [44]:
def create_sequences(df1, seq_length):
    X = []
    y = []
    for i in range(len(df1) - seq_length):
        X.append(df1[i:i+seq_length, 0])
        y.append(df1[i+seq_length, 1])
    return np.array(X), np.array(y)

In [45]:
seq_length = 12  
X_seq, y_seq = create_sequences(scaled_data, seq_length)
X_seq = np.reshape(X_seq, (X_seq.shape[0], X_seq.shape[1], 1))

In [46]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_seq.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

d:\PFE\libraires\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [47]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [48]:
model.fit(X_seq, y_seq, epochs=100, batch_size=32)

Epoch 1/100
23560/23560 ━━━━━━━━━━━━━━━━━━━━ 83s 3ms/step - loss: 8.4794e-05
Epoch 2/100
23560/23560 ━━━━━━━━━━━━━━━━━━━━ 77s 3ms/step - loss: 2.3523e-05
Epoch 3/100
23560/23560 ━━━━━━━━━━━━━━━━━━━━ 77s 3ms/step - loss: 2.3857e-05
Epoch 4/100
23560/23560 ━━━━━━━━━━━━━━━━━━━━ 77s 3ms/step - loss: 2.2819e-05
Epoch 5/100
23560/23560 ━━━━━━━━━━━━━━━━━━━━ 79s 3ms/step - loss: 2.3200e-05
Epoch 6/100
23560/23560 ━━━━━━━━━━━━━━━━━━━━ 77s 3ms/step - loss: 2.3571e-05
Epoch 7/100
23560/23560 ━━━━━━━━━━━━━━━━━━━━ 77s 3ms/step - loss: 2.5058e-05
Epoch 8/100
23560/23560 ━━━━━━━━━━━━━━━━━━━━ 77s 3ms/step - loss: 2.3271e-05
Epoch 9/100
23560/23560 ━━━━━━━━━━━━━━━━━━━━ 77s 3ms/step - loss: 2.2769e-05
Epoch 10/100
23560/23560 ━━━━━━━━━━━━━━━━━━━━ 77s 3ms/step - loss: 2.3818e-05
Epoch 11/100
23560/23560 ━━━━━━━━━━━━━━━━━━━━ 77s 3ms/step - loss: 2.2500e-05
Epoch 12/100
23560/23560 ━━━━━━━━━━━━━━━━━━━━ 78s 3ms/step - loss: 2.4925e-05
Epoch 13/100
23560/23560 ━━━━━━━━━━━━━━━━━━━━ 77s 3ms/step - loss: 2.3789

KeyboardInterrupt: 

In [49]:
predicted_data = model.predict(X_seq)

23560/23560 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step


In [50]:
predicted_data = scaler.inverse_transform(np.concatenate((X_seq[:, -1, :], predicted_data), axis=1))[:, 1]

In [57]:
predicted_data.sum()

31952380.555335056

In [54]:
rmse = mean_squared_error(df1.iloc[seq_length:, 1], predicted_data, squared=False)
print("RMSE:", rmse)

RMSE: 257.4031879740163


d:\PFE\libraires\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
